In [1]:
import solaris as sol
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import skimage
import pandas as pd
import geopandas as gpd
import rasterio
import torch
import scipy
from shapely.ops import cascaded_union  # just for visualization purposes

## Fine-tuning the model

### Creating training masks
Before we can continue training a model, we need target masks: images that the model will learn to create during training. We'll follow [this tutorial](https://solaris.readthedocs.io/en/latest/tutorials/notebooks/api_masks_tutorial.html) to create masks. __Note for workshop participants:__ this cell won't work because the `/data` directory is read-only; we've made the training masks for you, but this cell shows how to do it.

In [2]:
sol_path = '/home/wb447340/Notebooks/Imagery/solaris'
data_path_p = '/home/wb447340/Notebooks/Imagery/Feature_Extraction'
data_path_j = '/home/public/Data/COUNTRY/GHA/chippedbasemap_512'
data_path_train = '/home/public/Data/COUNTRY/GHA/TRAINING_DATA/RGB'
images = data_path_j + '/images'
masks = data_path_j + '/labels'
print(sol_path)
print(data_path_p)
print(data_path_j)
print(images)
print(masks)

/home/wb447340/Notebooks/Imagery/solaris
/home/wb447340/Notebooks/Imagery/Feature_Extraction
/home/public/Data/COUNTRY/GHA/chippedbasemap_512
/home/public/Data/COUNTRY/GHA/chippedbasemap_512/images
/home/public/Data/COUNTRY/GHA/chippedbasemap_512/labels


### Check if you can see your image chips, assemble the input list of chips as a CSV, and make sure they will be read in as a tensor 

In [ ]:
# Can you see the public drive with the data?
os.path.exists(data_path_j)
print(os.listdir(data_path_j))
os.path.exists(data_path_p)
print(os.listdir(data_path_p))
os.path.exists(data_path_train)
print(os.listdir(data_path_train))

# Create chips of imagery from COG tiles, then assemble a CSV

In [ ]:
inputTileFolder = "/home/public/Data/COUNTRY/GHA/basemap/raster_tiles/"
allImagetiles = []
for root, dirs, files in os.walk(inputTileFolder):
    for f in files:
        allImagetiles.append(os.path.join(root, f))
​
imgCnt = 0
​
for cImg in allImagetiles[:100]:
    imgCnt = imgCnt + 1
    if imgCnt > 100:
        break
    curR = rasterio.open(cImg)
    chipSize = 512
    tileName = os.path.basename(cImg).replace(".tif", "")
    print("Processing %s of 100: %s" % (imgCnt, tileName))
    
    
    baseOutFolder = "/home/public/Data/COUNTRY/GHA/chippedbasemap_%s" % chipSize
    outFolder = os.path.join(baseOutFolder, tileName)
    misOut = "%s_misshapen" % outFolder
    if not os.path.exists(outFolder):
        os.makedirs(outFolder)
    if not os.path.exists(misOut):
        os.makedirs(misOut)
​
    prof = {'driver': 'GTiff', 
            'dtype': 'uint8', 
            'nodata': None, 
            'count': 3,
            'StripOffsets': 1
    }
​
    xRange = list(range(0, curR.shape[0], chipSize)) + [curR.shape[0]]
    yRange = list(range(0, curR.shape[1], chipSize)) + [curR.shape[1]]
​
    allImages = []
    with rasterio.open(cImg) as src:
        prof.update(crs=src.crs)
        for xIdx in range(1, len(xRange)):
            for yIdx in range(1, len(yRange)):
                outFile = os.path.join(outFolder, "chip_%s_%s_%s.tif" % (tileName, xIdx, yIdx))            
                if not os.path.exists(outFile):
                    curWindow = Window.from_slices((xRange[xIdx - 1], xRange[xIdx]),
                                               (yRange[yIdx - 1], yRange[yIdx]))
                    temp = src.read([1,2,3], window=curWindow)
                    if temp.shape[1] != chipSize or temp.shape[2] != chipSize:
                        outFile = os.path.join(misOut, "chip_%s_%s.tif" % (xIdx, yIdx))
                    else:
                        allImages.append(outFile)
                        
                    prof.update(transform=transform(curWindow, transform=src.transform),
                            width=temp.shape[2], height=temp.shape[1])            
                    with rasterio.open(outFile, 'w', **prof) as dst:
                        dst.write(temp)
​
    with open("%s.csv" % outFolder, 'w') as fileList:
        cnt=0
        fileList.write(",image\n")
        for line in allImages:
            fileList.write("%s,%s\n" % (cnt, line.replace("/home/wb411133/data/Country","/home/public/Data/COUNTRY")))
            cnt += 1
​
# Combine csv files into one file
baseOutFolder ="/home/public/Data/COUNTRY/GHA/chippedbasemap_512"
allCsv = []
allTif = []
rgbImages = []
grayImages = []
mixedImages = []
for root, dirs, files in os.walk(baseOutFolder):
    for f in files:
        if f[-4:] == ".csv":
            allCsv.append(os.path.join(root, f))
        if f[-4:] == ".tif" and root[-6:] != "shapen" and root[-3:] != "nt8":
            fileName = os.path.join(root, f)
            allTif.append(fileName)
            #Calculate grayness of the image
            temp = rasterio.open(fileName).read()            
            sampleX = random.sample(range(0, temp.shape[1]), 10)
            sampleY = random.sample(range(0, temp.shape[2]), 10)
            pGray = 0
            for x in range(0,10):
                cTemp = temp[:,sampleX[x],sampleY[x]]
                if (cTemp[0] == cTemp[1] == cTemp[2]):
                    pGray += 1
            if pGray == 10:
                grayImages.append(fileName)
            elif pGray == 0:
                rgbImages.append(fileName)
            else:
                mixedImages.append(fileName)
                
cnt = 0
with open(os.path.join(baseOutFolder, "all_images_INF_GREY.csv"), 'w') as out:
    out.write(",image\n")
    for f in grayImages:
        out.write("%s,%s\n" % (cnt, f))
        cnt = cnt + 1
​
cnt = 0
with open(os.path.join(baseOutFolder, "all_images_INF_RGB.csv"), 'w') as out:
    out.write(",image\n")
    for f in rgbImages:
        out.write("%s,%s\n" % (cnt, f))
        cnt = cnt + 1
​
cnt = 0
with open(os.path.join(baseOutFolder, "all_images_INF_MIXED.csv"), 'w') as out:
    out.write(",image\n")
    for f in mixedImages:
        out.write("%s,%s\n" % (cnt, f))
        cnt = cnt + 1


### Take a look at the shape of in the input tensors that pytorch will read

In [ ]:
inCSV = os.path.join(data_path_train,'GHA_train_lnx.csv')
print(inCSV)
inD = pd.read_csv(inCSV)
for idx, row in inD.iterrows():
    try:
        cImg = skimage.io.imread(row['image'][1])
        print(cImg.shape)
    except:
        print(row['image'])
print(inD)

In [ ]:
# Can you import your Multispectral input chips?
snapshot_RGB = skimage.io.imread('//home/public/Data/COUNTRY/GHA/TRAINING_DATA/RGB/images/000000000.tif')
snapshot_PAN = skimage.io.imread('//home/public/Data/COUNTRY/GHA/TRAINING_DATA/RGB/labels/000000000.tif')
x = torch.tensor(snapshot_RGB)
y = torch.tensor(snapshot_PAN)
print(x.shape)
print(y.shape)
print(y.dtype)

In [ ]:
# Visulaize your input chips
f, axarr = plt.subplots(figsize=(10, 10))
#torch.type(snapshot_RGB)
plt.imshow(snapshot_RGB, cmap='gray')
f, axarr = plt.subplots(figsize=(10, 10))
plt.imshow(snapshot_PAN, cmap='gray')


### (Optional) If you need to convert Signed 16 bit to unsigned 8 bit

In [ ]:
import os
import rasterio
inFiles = []

for root, dirs,  files in os.walk(data_path_chips):
    for f in files:
        if f[-4:] == '.tif':
            inFiles.append(os.path.join(root, f))

outFolder = data_path_chips + '/8uint8'

if not os.path.exists(outFolder):
    os.mkdir(outFolder)


for f in inFiles:
    curF = rasterio.open(f)
    curD = curF.read()[0,:,:].astype('uint8')
    curD = (curD > 0) * 1
    curD = curD.astype('uint8')
    profile = curF.profile
    profile.update(count=1, dtype='uint8')
    outFile = os.path.join(outFolder, os.path.basename(f))
    with rasterio.open(outFile, 'w', **profile) as dst:
        dst.write_band(1, curD)

### (Optional) If you need to covert .Shp to GEOJSON then to Binary Mask

In [ ]:
inBuildings = os.path.join(data_path_p, 'Buildings.shp')
gdf2 = gpd.read_file(inBuildings)
badIdx = []
for idx, row in gdf2.iterrows():
   try:
       xx = row['geometry'].area
   except:
       badIdx.append(idx)
gdf2 = gdf2.drop(badIdx)
fp_mask = sol.vector.mask.footprint_mask(gdf2, reference_im=os.path.join(data_path_j, 'raster_tiles/031111332333.tif'))
f, ax = plt.subplots(figsize=(10, 10))
plt.imshow(fp_mask, cmap='gray')

### (Optional) If you need another way to do the same thing (e.g. GEOJSON to .tiff mask)

In [ ]:
##make sure that you do not have any blank rows in your CSV referenced in the .yml
geojson_dir = os.path.join(data_path, 'geojson')
geojson_list = [f for f in os.listdir(geojson_dir) if f.endswith('.geojson')]
im_list = [f for f in os.listdir(geojson_dir) if f.endswith('.tif')]
n_chips = len(geojson_list)

if not os.path.exists(masks):
    os.mkdir(mask_dir)
    
    for idx, gj in enumerate(geojson_list):
        # get the 'img[number] chip ID for the image'
        chip_id = os.path.splitext(gj)[0].split('_')[-1]
        matching_im = chip_id + '.tif'
        mask_fname = 'masks' + chip_id + '.tif'
        fp_mask = sol.vector.mask.footprint_mask(df=os.path.join(geojson_dir, gj),
                                                 out_file=os.path.join(masks, mask_fname),
                                                 reference_im=os.path.join(im_dir, matching_im),
                                                 shape=(650, 650))
        if (idx+1)%100 == 0:
            print('chip {} of {} done'.format(idx+1, n_chips), flush=True)

Looks good! We're ready to set up for training.

### Building the config file

With model fine-tuning, we'll load the pre-trained weights used above, and continue training at a much lower learning rate for a couple of epochs. To this end we'll need _another_ config with a few more modifications:

1. A reduced learning rate - we'll try `1e-5` instead of `1e-4`
2. Change `train=False` to `train=True`
3. Specify where the newly trained versions are saved with the `training['callbacks']['model_checkpoint']` arguments
4. Specify a training data CSV. In this case, we'll use a CSV created [per this tutorial](https://solaris.readthedocs.io/en/latest/tutorials/notebooks/creating_im_reference_csvs.html) that points to all of the images and the masks that we just created, save for one: the image that we inferenced against earlier, which we'll save as a test image. The csv, named `khartoum_fine_tune.csv`, is available in the `workshop_configs` directory.

As earlier, feel free to create this config yourself; otherwise, you can use `xdxd_workshop_khartoum_train.yml`.

### Model training

Let's try it! <font style="color: red;">__WARNING: this is EXTREMELY slow without a GPU (each epoch may take several hours).__</font>

### Train the model - Your .yml contains all the relevant parameters

In [ ]:
print('Loading config...')
config = sol.utils.config.parse('/home/wb447340/Notebooks/Imagery/Feature_Extraction/GHA_RGB_train.yml')
print('config loaded. Initializing Trainer instance...')
xdxd_trainer = sol.nets.train.Trainer(config)
print('model initialized. Beginning training...')
print()
start_time = time.time()
xdxd_trainer.train()
end_time = time.time()
print()
print('training took {} minutes'.format((end_time-start_time)/60))

Loading config...
config loaded. Initializing Trainer instance...
model initialized. Beginning training...

Beginning training epoch 0
    loss at batch 0: 13.58376693725586
    loss at batch 10: 13.813175201416016
    loss at batch 20: 13.303665161132812
    loss at batch 30: 13.779580116271973
    loss at batch 40: 13.586511611938477
    loss at batch 50: 13.071934700012207
    loss at batch 60: 13.755105018615723
    loss at batch 70: 13.624524116516113
    loss at batch 80: 13.920515060424805
    loss at batch 90: 13.080217361450195


If you got a CUDA out of memory error in the cell above, kill the kernels for the other jupyter notebooks (instructions at the top of this notebook for how to do so), re-load this notebook, and try again. Note that you'll need to re-run the first cell of this notebook (all of the imports) before you'll be able to run this one again.

### Predictions with the new model

We'll now run inference with the newly tuned model. Note that if your config file specifies `train=True` and you pass that config to an `Inferer` instance, `solaris` will automatically use the newly trained model for inference.

In [ ]:
torch.cuda.is_available()
#torch.cuda.set_device(0)

In [ ]:
print('Loading config...')
config = sol.utils.config.parse(os.path.join(data_path_p, 'GHA_RGB_inf.yml'))
print('config loaded. Initializing model...')
xdxd_inferer = sol.nets.infer.Inferer(config)
print('model initialized. Loading dataset...')
inf_df = sol.nets.infer.get_infer_df(config)
print('dataset loaded. Running inference on the image.')
start_time = time.time()
xdxd_inferer(inf_df)
end_time = time.time()
print('running inference on one image took {} seconds'.format(end_time-start_time))
print('now go transform the output to vector...')

### Vectorize the outputs

In [ ]:
outFolder = 'PREDICTIONS_VECTOR'
if not os.path.exists(outFolder):
   os.makedirs(outFolder)
   
for curImg in inf_df['image']:
   predictionImage = os.path.join(data_path_p, 'PREDICTIONS_RGB', os.path.basename(curImg))
   resulting_preds = skimage.io.imread(predictionImage)
   resulting_preds = resulting_preds > 0
   predicted_footprints = sol.vector.mask.mask_to_poly_geojson(
       pred_arr=resulting_preds,
       reference_im=curImg,
       do_transform=True,
       min_area=1e-10
   )
   try:
       outFile = os.path.join(outFolder, os.path.basename(curImg).replace(".tif", ".json"))
       predicted_footprints.to_file(outFile, driver='GeoJSON')
   except:
       print('No footprints for %s' % predictionImage)
fp_mask = sol.vector.mask.footprint_mask('chip_1_10.json')
f, ax = plt.subplots(figsize=(10, 10))
plt.imshow(fp_mask, cmap='gnuplot')

In [ ]:
f, axarr = plt.subplots(2, 2, figsize=(12, 8))
axarr[0, 0].imshow(im_arr)
axarr[0, 0].set_title('Source image', size=14)
axarr[0, 0].axis('off')
axarr[0, 1].imshow(old_preds, cmap='gray')
axarr[0, 1].set_title('Predictions before fine-tuning', size=14)
axarr[0, 1].axis('off')
axarr[1, 1].imshow(new_preds, cmap='gray')
axarr[1, 1].set_title('Predictions after fine-tuning', size=14)
axarr[1, 1].axis('off')
axarr[1, 0].imshow(ground_truth, cmap='gray')
axarr[1, 0].set_title('Ground Truth', size=14)
axarr[1, 0].axis('off');

Wow. This appears to show a _marked_ improvement with _just three epochs of training!_ How do the scores come out?

## Scoring model performance after fine-tuning

In [ ]:
evaluator = sol.eval.base.Evaluator(os.path.join(data_path_p, 'Pred_RGB.geojson'))
prediction_dirs = ['inference_out', 'retrain_inference_out']
model_names = ['Original', 'Fine-tuned']

f1_scores = []
precision = []
recall = []
for i in range(2):
    evaluator.load_proposal(os.path.join(prediction_dirs[i],'Pred_RGB.geojson'),
                            pred_row_geo_value='geometry',
                            conf_field_list=[])
    results = evaluator.eval_iou(miniou=0.5, calculate_class_scores=False)
    f1_scores.append(results[0]['F1Score'])
    precision.append(results[0]['Precision'])
    recall.append(results[0]['Recall'])

f, axarr = plt.subplots(1, 3, figsize=(10, 4))
f.subplots_adjust(wspace=0.6)
axarr[0].bar(model_names, f1_scores)
axarr[0].set_ylabel('$F_1$ Score', size=16)
axarr[1].bar(model_names, precision)
axarr[1].set_ylabel('Precision', size=16)
axarr[2].bar(model_names, recall)
axarr[2].set_ylabel('Recall', size=16);
f.suptitle('Comparison of original vs. fine-tuned model performance', size=16);

Clearly, this is only _one_ sample image; however, it's noteworthy that this model briefly fine-tuned on Khartoum imagery [__achieved a higher score here than some of the prize-winning models trained on Khartoum for days during the SpaceNet Challenge Round 2__](https://medium.com/the-downlinq/2nd-spacenet-competition-winners-code-release-c7473eea7c11).

# Congratulations! You've completed the FOSS4G 2019 Solaris tutorial.

Hang around for a quick teaser on the SpaceNet 5 challenge that's starting soon! You're also welcome to explore the documentation or install solaris on your own machine and play around! We'll be here till the workshop ends and able to help.

## What's next?

Here are a few more resources that will help you as you continue to work with `solaris`:

- [Solaris documentation](https://solaris.readthedocs.io)
- [A blog post from Jake Shermeyer about using Solaris for car detection in the COWC dataset](https://medium.com/the-downlinq/beyond-infrastructure-mapping-finding-vehicles-with-solaris-11e08da0dab)
- [The Solaris GitHub repository](https://github.com/cosmiq/solaris)